In [1]:
import random
import math

# -------------------------------
# Parameters
# -------------------------------
NUM_ANTS = 5
NUM_ITERATIONS = 10
ALPHA = 1       # pheromone importance
BETA = 2        # heuristic importance
RHO = 0.5       # evaporation rate
Q = 100         # pheromone deposit factor

# Network represented as adjacency matrix (costs)
# 0 means no direct connection
graph = [
    [0, 2, 0, 1, 0],
    [2, 0, 3, 2, 0],
    [0, 3, 0, 0, 2],
    [1, 2, 0, 0, 3],
    [0, 0, 2, 3, 0]
]

num_nodes = len(graph)
pheromone = [[1 for _ in range(num_nodes)] for _ in range(num_nodes)]

source = 0
destination = 4

# -------------------------------
# Helper functions
# -------------------------------

def heuristic(i, j):
    """Heuristic value: inverse of cost"""
    if graph[i][j] == 0:
        return 0
    return 1 / graph[i][j]

def select_next_node(current, visited):
    """Select next node probabilistically based on pheromone and heuristic"""
    probs = []
    total = 0
    for j in range(num_nodes):
        if graph[current][j] != 0 and j not in visited:
            tau = pheromone[current][j] ** ALPHA
            eta = heuristic(current, j) ** BETA
            total += tau * eta
            probs.append((j, tau * eta))
    if not probs:
        return None

    # Normalize probabilities
    r = random.random()
    cumulative = 0
    for j, p in probs:
        cumulative += p / total
        if r <= cumulative:
            return j
    return probs[-1][0]

def route_cost(path):
    """Compute total cost of the route"""
    cost = 0
    for i in range(len(path) - 1):
        cost += graph[path[i]][path[i + 1]]
    return cost

# -------------------------------
# Main ACO Loop
# -------------------------------
best_path = None
best_cost = math.inf

for iteration in range(NUM_ITERATIONS):
    all_paths = []
    all_costs = []

    for ant in range(NUM_ANTS):
        visited = [source]
        current = source

        # Construct path
        while current != destination:
            next_node = select_next_node(current, visited)
            if next_node is None:
                break  # dead end
            visited.append(next_node)
            current = next_node

        if visited[-1] == destination:
            cost = route_cost(visited)
            all_paths.append(visited)
            all_costs.append(cost)

    # Update best path
    if all_costs:
        min_cost = min(all_costs)
        min_index = all_costs.index(min_cost)
        if min_cost < best_cost:
            best_cost = min_cost
            best_path = all_paths[min_index]

    # Pheromone evaporation
    for i in range(num_nodes):
        for j in range(num_nodes):
            pheromone[i][j] *= (1 - RHO)

    # Pheromone deposit
    for path, cost in zip(all_paths, all_costs):
        for i in range(len(path) - 1):
            a, b = path[i], path[i + 1]
            pheromone[a][b] += Q / cost
            pheromone[b][a] = pheromone[a][b]

    # -------------------------------
    # Display progress
    # -------------------------------
    print(f"Iteration {iteration + 1}:")
    for i, path in enumerate(all_paths):
        print(f"  Ant {i + 1} path: {path} | cost = {all_costs[i]}")
    print(f"  Best path so far: {best_path} | cost = {best_cost}\n")

# -------------------------------
# Final Output
# -------------------------------
print("===================================")
print("Final Best Route Found:")
print(f"Path: {best_path}")
print(f"Total Cost: {best_cost}")
print("===================================")


Iteration 1:
  Ant 1 path: [0, 3, 4] | cost = 4
  Ant 2 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 4 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 5 path: [0, 1, 3, 4] | cost = 7
  Best path so far: [0, 3, 4] | cost = 4

Iteration 2:
  Ant 1 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 2 path: [0, 1, 3, 4] | cost = 7
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 4 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 5 path: [0, 1, 3, 4] | cost = 7
  Best path so far: [0, 3, 4] | cost = 4

Iteration 3:
  Ant 1 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 2 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 4 path: [0, 1, 3, 4] | cost = 7
  Ant 5 path: [0, 3, 1, 2, 4] | cost = 8
  Best path so far: [0, 3, 4] | cost = 4

Iteration 4:
  Ant 1 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 2 path: [0, 3, 4] | cost = 4
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 8
  Ant 4 path: [0, 3, 4] | cost = 4
  Ant 5 path: [0, 3, 1, 2, 4] | cost = 8
  Best path so far: [0, 3, 4] | 

In [6]:
import random
import math

# -------------------------------
# Parameters
# -------------------------------
NUM_ANTS = 6             # Number of packets/ants
NUM_ITERATIONS = 10      # Maximum iterations
ALPHA = 1                # Importance of pheromone
BETA = 2                 # Importance of heuristic (1/distance)
RHO = 0.5                # Evaporation rate
Q = 100                  # Pheromone deposit factor

# -------------------------------
# Network Topology (Adjacency Matrix)
# 0 means no direct link
# Each value represents link cost (e.g., delay, distance, etc.)
# -------------------------------
graph = [
    [0, 2, 0, 1, 0],
    [2, 0, 3, 1, 0],
    [0, 3, 0, 0, 2],
    [1, 1, 0, 0, 3],
    [0, 0, 4, 3, 0]
]

num_nodes = len(graph)
pheromone = [[1 for _ in range(num_nodes)] for _ in range(num_nodes)]

source = 0
destination = 4

# -------------------------------
# Helper Functions
# -------------------------------

def heuristic(i, j):
    """Heuristic value: inverse of cost"""
    if graph[i][j] == 0:
        return 0
    return 1 / graph[i][j]

def select_next_node(current, visited):
    """Select next node based on pheromone and heuristic"""
    probabilities = []
    total = 0
    for j in range(num_nodes):
        if graph[current][j] != 0 and j not in visited:
            tau = pheromone[current][j] ** ALPHA
            eta = heuristic(current, j) ** BETA
            total += tau * eta
            probabilities.append((j, tau * eta))
    
    if not probabilities:
        return None
    
    # Roulette wheel selection
    r = random.random()
    cumulative = 0
    for node, prob in probabilities:
        cumulative += prob / total
        if r <= cumulative:
            return node
    return probabilities[-1][0]

def route_cost(path):
    """Compute total cost of a given route"""
    cost = 0
    for i in range(len(path) - 1):
        cost += graph[path[i]][path[i + 1]]
    return cost

# -------------------------------
# ACO Main Loop
# -------------------------------
best_path = None
best_cost = math.inf

for iteration in range(NUM_ITERATIONS):
    all_paths = []
    all_costs = []

    for ant in range(NUM_ANTS):
        visited = [source]
        current = source

        # Build path
        while current != destination:
            next_node = select_next_node(current, visited)
            if next_node is None:
                break  # Dead-end
            visited.append(next_node)
            current = next_node

        if visited[-1] == destination:
            cost = route_cost(visited)
            all_paths.append(visited)
            all_costs.append(cost)

    # Update global best
    if all_costs:
        min_cost = min(all_costs)
        min_index = all_costs.index(min_cost)
        if min_cost < best_cost:
            best_cost = min_cost
            best_path = all_paths[min_index]

    # Evaporate pheromone
    for i in range(num_nodes):
        for j in range(num_nodes):
            pheromone[i][j] *= (1 - RHO)

    # Deposit pheromone
    for path, cost in zip(all_paths, all_costs):
        for i in range(len(path) - 1):
            a, b = path[i], path[i + 1]
            pheromone[a][b] += Q / cost
            pheromone[b][a] = pheromone[a][b]  # symmetric update

    # Display iteration progress
    print(f"Iteration {iteration + 1}:")
    if all_costs:
        for idx, path in enumerate(all_paths):
            print(f"  Ant {idx + 1} path: {path} | cost = {all_costs[idx]}")
        print(f"  >> Best path so far: {best_path} | cost = {best_cost}")
    else:
        print("  No valid paths found in this iteration.")
    print("-" * 50)

# -------------------------------
# Final Output
# -------------------------------
print("\nFINAL RESULT")
print("=" * 50)
print(f"Best Route Found: {best_path}")
print(f"Total Cost: {best_cost}")
print("=" * 50)


Iteration 1:
  Ant 1 path: [0, 1, 3, 4] | cost = 6
  Ant 2 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 4 path: [0, 1, 3, 4] | cost = 6
  Ant 5 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 6 path: [0, 3, 1, 2, 4] | cost = 7
  >> Best path so far: [0, 1, 3, 4] | cost = 6
--------------------------------------------------
Iteration 2:
  Ant 1 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 2 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 4 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 5 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 6 path: [0, 3, 1, 2, 4] | cost = 7
  >> Best path so far: [0, 1, 3, 4] | cost = 6
--------------------------------------------------
Iteration 3:
  Ant 1 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 2 path: [0, 1, 3, 4] | cost = 6
  Ant 3 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 4 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 5 path: [0, 3, 1, 2, 4] | cost = 7
  Ant 6 path: [0, 1, 3, 4] | cost = 6
  >> Best path so far: [0, 1, 3, 4] | c

In [7]:
import numpy as np
import random
import math

# ===============================
# Ant Colony Optimization for TSP
# ===============================

class ACO_TSP:
    def __init__(self, num_ants, num_iterations, alpha, beta, rho, cities):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha              # Influence of pheromone
        self.beta = beta                # Influence of heuristic (1/distance)
        self.rho = rho                  # Evaporation rate
        self.cities = cities
        self.num_cities = len(cities)

        # Distance matrix
        self.dist_matrix = self.compute_distances()
        # Initial pheromone on all edges
        self.pheromone = np.ones((self.num_cities, self.num_cities))
        
        # Keep track of best solution
        self.best_length = float('inf')
        self.best_tour = None

    def compute_distances(self):
        """Compute Euclidean distance matrix between cities."""
        n = len(self.cities)
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    dist_matrix[i][j] = math.sqrt((self.cities[i][0] - self.cities[j][0])**2 +
                                                  (self.cities[i][1] - self.cities[j][1])**2)
        return dist_matrix

    def probability(self, i, visited):
        """Compute probability of moving from city i to all other cities."""
        pheromone = np.copy(self.pheromone[i])
        heuristic = 1 / (self.dist_matrix[i] + 1e-10)
        
        pheromone[visited] = 0  # Zero out visited cities
        heuristic[visited] = 0

        numerator = (pheromone ** self.alpha) * (heuristic ** self.beta)
        if numerator.sum() == 0:
            return np.ones(len(pheromone)) / len(pheromone)
        prob = numerator / numerator.sum()
        return prob

    def construct_solution(self):
        """Each ant constructs a path based on pheromone and heuristic."""
        path = []
        start = random.randint(0, self.num_cities - 1)
        path.append(start)

        while len(path) < self.num_cities:
            current = path[-1]
            prob = self.probability(current, path)
            next_city = np.random.choice(range(self.num_cities), p=prob)
            path.append(next_city)

        return path

    def path_length(self, path):
        """Compute total length of a path."""
        length = 0
        for i in range(len(path) - 1):
            length += self.dist_matrix[path[i]][path[i+1]]
        length += self.dist_matrix[path[-1]][path[0]]  # Return to start
        return length

    def update_pheromone(self, all_paths):
        """Update pheromone trails."""
        self.pheromone *= (1 - self.rho)  # Evaporation

        for path, length in all_paths:
            for i in range(len(path) - 1):
                a, b = path[i], path[i + 1]
                self.pheromone[a][b] += 1.0 / length
                self.pheromone[b][a] += 1.0 / length  # symmetric

    def run(self):
        """Main loop for ACO algorithm."""
        for iteration in range(self.num_iterations):
            all_paths = []

            for ant in range(self.num_ants):
                path = self.construct_solution()
                length = self.path_length(path)
                all_paths.append((path, length))

                if length < self.best_length:
                    self.best_length = length
                    self.best_tour = path

            self.update_pheromone(all_paths)

            print(f"Iteration {iteration+1}: Best path length = {self.best_length:.4f}")

        return self.best_tour, self.best_length


# =================================
# Example Run
# =================================
if __name__ == "__main__":
    # Define city coordinates (x, y)
    cities = [
        (0, 0), (1, 5), (5, 2), (6, 6), (8, 3),
        (7, 9), (2, 7), (3, 3), (9, 8), (4, 9)
    ]

    aco = ACO_TSP(
        num_ants=10,
        num_iterations=10,
        alpha=1.0,
        beta=5.0,
        rho=0.5,
        cities=cities
    )

    best_tour, best_length = aco.run()

    print("\nBest Tour Found:", best_tour)
    print("Shortest Distance:", best_length)


Iteration 1: Best path length = 33.9074
Iteration 2: Best path length = 33.9074
Iteration 3: Best path length = 32.2517
Iteration 4: Best path length = 32.2517
Iteration 5: Best path length = 32.2517
Iteration 6: Best path length = 32.2517
Iteration 7: Best path length = 32.2517
Iteration 8: Best path length = 32.2517
Iteration 9: Best path length = 32.2517
Iteration 10: Best path length = 32.2517

Best Tour Found: [1, 6, 9, 5, 8, 3, 4, 2, 7, 0]
Shortest Distance: 32.25167146905398
